# Tutorial: Coulomb Matrix

The Coulomb Matrix representation was designed for the modeling of atomization energies and as a result requires information needed by the Hamiltonian, namely the set of Cartesian coordinates $\left\{ \mathbf { R } _ { I } \right\}$ and nuclear charges $\left\{ Z _ { I } \right\}$. With this information the Coulomb matrix $\mathbf{M}$ is built

$$M _ { I J } = \left\{ \begin{array} { l l } { 0.5 Z _ { I } ^ { 2.4 } } & { \text { for } I = J } \\ { \frac { Z _ { I } Z _ { J } } { \left| \mathbf { R } _ { I } - \mathbf { R } _ { J } \right| } } & { \text { for } I \neq J } \end{array} \right. $$

where the off-diagonal elements correspond to the Coulomb repulsion between $I$ and $J$. The expression for the diagonal elements is the result of an encoded polynomial fit to the atomic energies and nuclear charge. 

    - DOI: 10.1103/PhysRevLett.108.058301
    
We begin by importing the Coulomb matrix constructor from `chemreps.coulomb_matrix` 

In [ ]:
from chemreps.coulomb_matrix import coulomb_matrix
import pandas as pd
import numpy as np
import glob

As an example, we will build the Coulomb matrix for butane. The data set that we will be using can be found in the data directory of this repository. If you cloned this repository locally, then you should be able to set the path as '../data/sdf/'.

The matrix is built by calling the imported `coulomb_matrix` function. The first argument is the coordinate file of the molecule of interest. The second argument is the size of the matrix, which should be the number of atoms in the molecule. An error is raised if the size given is not large enough.



In [ ]:
mfile = '../data/sdf/butane.sdf'
cm = coulomb_matrix(mfile, size=14)

The matrix we have now created is stored in the variable cm.

In [ ]:
print(cm)

**Note:** As you run on a set of molecules to build your training set, the Coulomb matrices must be of the same size. To accomplish this matrices of smaller molecules are padded with zeros until they are the size of the largest molecule's Coulomb matrix. We will see this in the example below.

### Making Representations for Multiple Molecules

Disclaimer: There may be better ways to accomplish the same objective. You are welcome to use your method as well as submit a issue/PR if you think we should use that method.

To make representations for all the molecules in our directory we are going to need to use glob and pandas. To find out more about these libraries you can go to the [glob documentation](https://docs.python.org/3/library/glob.html) or [10 Minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html). We are going to first create an empty list called rep_list in which we will store information such as the filename and the representation. Next we loop over all of the files in the directory using glob to match our pattern (eg. we want all sdf files from our data/sdf/ directory). In this loop we use the same method as above in order to make our representations. We store the name of the file and the representation in a dictionary that is then appended to our rep_list. Once the loop is complete, we store the information in a pandas dataframe.

As you can see below, the variable `size_largest_mol` represents the size of the largest molecule in the set. For our example data, the largest molecule is penicillin with 42 molecules.

In [ ]:
dataset = '../data/sdf/'

row_list = []
size_of_largest = 42
for i in sorted(glob.iglob(dataset + '/*')):
    fname = i
    print(fname)
    rep = coulomb_matrix(fname, size=size_of_largest)
    dict1 = {}
    dict1.update({'Name': fname})
    dict1.update({'Rep': rep})
    row_list.append(dict1)

df = pd.DataFrame(row_list, columns=['Name', 'Rep'])
df

Once our representation information is stored in the pandas dataframe, we can use numpy in order to make an array of our representations that we can finally pass to our machine learning method.

In [ ]:
reps = np.asarray(df['Rep'])
reps